In [14]:
import mlflow
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import warnings

In [15]:
remote_server_uri = "http://127.0.0.1:8080"
mlflow.set_tracking_uri(remote_server_uri)

In [5]:
mlflow.tracking.get_tracking_uri()

'http://127.0.0.1:8080'

In [16]:
# Define evaluation metrics function
def eval_metrics(actual, pred):
    rmse = np.sqrt(mean_squared_error(actual, pred))
    mae = mean_absolute_error(actual, pred)
    r2 = r2_score(actual, pred)
    return rmse, mae, r2

# Define data loading function
def load_data(data_path):
    data = pd.read_csv(data_path)
    train, test = train_test_split(data, test_size=0.25, random_state=42)
    train_x = train.drop(["median_house_value"], axis=1)
    test_x = test.drop(["median_house_value"], axis=1)
    train_y = train[["median_house_value"]]
    test_y = test[["median_house_value"]]
    return train_x, train_y, test_x, test_y

# Create experiment and start the parent run
experiment_id = mlflow.create_experiment("Housing_Price_Linear_Regression")
with mlflow.start_run(run_name="Linear_Regression_PR", experiment_id=experiment_id,
                      tags={"version": "v1", "priority": "P1"}, description="Data Preprocessing") as parent_run:
    # Data preprocessing
    dh = pd.read_csv("../datasets/housing/housing.csv")
    # Perform preprocessing steps here...

    # Load and split data
    warnings.filterwarnings("ignore")
    np.random.seed(40)
    data_path = "../datasets/housing/housing_clean.csv"
    train_x, train_y, test_x, test_y = load_data(data_path)
    mlflow.log_param("Data Split", "yes")

    # Start child run for modeling
    with mlflow.start_run(run_name="MODELLING", nested=True, description="Modelling"):
        # Model training and evaluation
        lr = LinearRegression()
        lr.fit(train_x, train_y)

        # Evaluate Metrics
        predicted_qualities = lr.predict(test_x)
        rmse, mae, r2 = eval_metrics(test_y, predicted_qualities)

        # Log metrics and artifacts
        print("Linear Regression  model:")
        print("  RMSE: %s" % rmse)
        print("  MAE: %s" % mae)
        print("  R2: %s" % r2)
        mlflow.log_metric("rmse", rmse)
        mlflow.log_metrics({"mae": mae, "r2": r2})
        mlflow.log_artifact(data_path)
        print("Save to: {}".format(mlflow.get_artifact_uri()))
        mlflow.sklearn.log_model(lr, "model")
        mlflow.log_param("Modelling", "yes")

    # Start child run for child specific logging
    with mlflow.start_run(run_name="CHILD_RUN", nested=True, description="child"):
        mlflow.log_param("child_1", "yes")

# Print parent run details
print("parent run:")
print("run_id: {}".format(parent_run.info.run_id))
print("description: {}".format(parent_run.data.tags.get("mlflow.note.content")))
print("version tag value: {}".format(parent_run.data.tags.get("version")))
print("priority tag value: {}".format(parent_run.data.tags.get("priority")))


Linear Regression  model:
  RMSE: 0.5919929314031497
  MAE: 0.4278697585966107
  R2: 0.64733757899314
Save to: /home/saikiran/mle-training/mlruns/0/d69f7646082646868d818b78c17d3fb9/artifacts
parent run:
run_id: 343eb671e13848b7bc2fe45700ae3def
description: Data Preprocessing
version tag value: v1
priority tag value: P1
